In [1]:
import codecs
import csv
import time
import os
import re
import gzip
import pandas as pd
import numpy as np

In [2]:
PATH_TO_DATA = 'Data/'
PATH_TO_DATA_EN = PATH_TO_DATA+"enwiki/"
PATH_TO_DATA_UK = PATH_TO_DATA+"ukwiki/"

In [3]:
ENWIKI_ART_FNMS = []
for file in os.listdir(PATH_TO_DATA_EN):
    if re.match(r"enwiki-20180620-pages-meta-current\d{2}-p\d+p\d+.xml_art.csv.gz", file):
        ENWIKI_ART_FNMS.append(file)
ENWIKI_RED_FNMS = []
for file in os.listdir(PATH_TO_DATA_EN):
    if re.match(r"enwiki-20180620-pages-meta-current\d{2}-p\d+p\d+.xml_red.csv.gz", file):
        ENWIKI_RED_FNMS.append(file)        

In [4]:
def unpack(file_name):
    file_name_new = file_name.replace(".gz","")
    with gzip.open(file_name, 'rb') as f_in, open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    return file_name_new

In [5]:
def pack_and_remove(file_name):
    file_name_new = file_name+'.gz'
    with open(file_name, 'rb') as f_in, gzip.open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(file_name)
    return file_name_new

In [6]:
df_title_count = None
for fn in ENWIKI_ART_FNMS:
    fn = PATH_TO_DATA_EN+fn
    print(fn)
    fn_new = unpack(fn)
    df_articles = pd.read_csv(fn_new, encoding='ISO-8859-1',quotechar="'")

    df_red_links = df_articles[df_articles['is_red_link'] == True]
    df_title_count_tmp = pd.DataFrame(df_red_links.groupby('link_val').link_val.count())
    df_title_count_tmp.columns = ['link']
    df_title_count_tmp = df_title_count_tmp.reset_index(col_level=1)
    df_title_count_tmp.columns = ['link_title', 'in_count']
    
    if df_title_count is not None:
        df_title_count = df_title_count.append(df_title_count_tmp)        
        #print("append")
    else:
        df_title_count = df_title_count_tmp
        #print("assign")    
    df_title_count = df_title_count.groupby('link_title').in_count.sum().copy()
    df_title_count.columns = ['link']
    df_title_count = df_title_count.reset_index()
    df_title_count.columns = ['link_title', 'in_count']        
    print("df_title_count size: {}".format(df_title_count.shape))
    os.remove(fn_new)

Data/enwiki/enwiki-20180620-pages-meta-current01-p10p30303.xml_art.csv.gz
df_title_count size: (26887, 2)
Data/enwiki/enwiki-20180620-pages-meta-current02-p30304p88444.xml_art.csv.gz
df_title_count size: (73462, 2)
Data/enwiki/enwiki-20180620-pages-meta-current03-p88445p200507.xml_art.csv.gz
df_title_count size: (146899, 2)
Data/enwiki/enwiki-20180620-pages-meta-current04-p200511p352689.xml_art.csv.gz
df_title_count size: (246012, 2)
Data/enwiki/enwiki-20180620-pages-meta-current05-p352690p565313.xml_art.csv.gz
df_title_count size: (368615, 2)
Data/enwiki/enwiki-20180620-pages-meta-current06-p565314p892912.xml_art.csv.gz
df_title_count size: (492089, 2)
Data/enwiki/enwiki-20180620-pages-meta-current07-p892914p1268691.xml_art.csv.gz
df_title_count size: (631762, 2)
Data/enwiki/enwiki-20180620-pages-meta-current08-p1268692p1791079.xml_art.csv.gz
df_title_count size: (786642, 2)
Data/enwiki/enwiki-20180620-pages-meta-current09-p1791080p2336422.xml_art.csv.gz
df_title_count size: (944630, 

In [7]:
df_title_count = df_title_count.sort_values(['in_count'], ascending=[0])

In [8]:
df_title_count.to_csv(PATH_TO_DATA_EN+'enwiki-20180620-red_name_count.csv', index = False, encoding='ISO-8859-1', quotechar="'", escapechar ="\\")

In [9]:
df_title_count

,link_title,in_count
36,,6572
5684449,]],6023
3096939,List of South Korean films of 19xx,4451
5789221,{{Fb position,2530
5107262,The Poll,1547
4086564,"Philippine general election, {{PH wikidata",1463
5789234,{{PH wikidata,1420
4528516,Saaremaa Parish,841
5789251,{{fb position,633
5789263,|,620


In [10]:
df_title_count.describe()

,in_count
count,5.826322e+06
mean,1.671689e+00
std,5.195138e+00
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,2.000000e+00
max,6.572000e+03


In [11]:
df_count_by_count = pd.DataFrame(df_title_count.groupby('in_count').in_count.count())

In [12]:
df_count_by_count.columns = ['link']
df_count_by_count = df_count_by_count.reset_index()
df_count_by_count.columns = ['count', 'in_count']  

In [13]:
df_count_by_count = df_count_by_count.sort_values(['count'], ascending=[1])

In [14]:
df_count_by_count

,count,in_count
0,1,4365576
1,2,813513
2,3,268516
3,4,130033
4,5,69864
5,6,44644
6,7,29389
7,8,20781
8,9,15277
9,10,11658


In [15]:
df_count_by_count.describe()

,count,in_count
count,236.000000,2.360000e+02
mean,234.245763,2.468781e+04
std,673.407106,2.893426e+05
min,1.000000,1.000000e+00
25%,59.750000,1.000000e+00
50%,118.500000,6.000000e+00
75%,189.250000,6.275000e+01
max,6572.000000,4.365576e+06


In [16]:
df_count_by_count.to_csv(PATH_TO_DATA_EN+'enwiki-20180620-red_count_by_count.csv', index = False, encoding='ISO-8859-1', quotechar="'", escapechar ="\\")